In [101]:
import pandas as pd
import json

# TODO :

* market basket analysis: pour determiner les corelations entre les evenements
* Variable Importance with ML Decision Tree: separer les donées en n classe pour variable interreser. n étant les velurs unique dans la classe et evaluer l'importance des valeurs

In [102]:
def add_cols(df: pd.DataFrame, cols: list, init_val):
    for col in cols:
        df[str(col)] = init_val
        
    return df


def prefix_with_col(df: pd.DataFrame) -> pd.DataFrame:
    result = add_cols(pd.DataFrame(), df.columns, '')
    result['Num_Acc'] = df['Num_Acc']
    for col in df.columns:
        if str(col) == 'Num_Acc': continue
        result[str(col)] = df[col].map(lambda x: f'{col}__{convert_to_int(x)}')
            
    return result

In [103]:
data_folder = '../data/accidents-in-france-from-2005-to-2016/'

caracteristics = pd.read_csv(data_folder + 'caracteristics.csv', encoding='latin-1')

/opt/miniconda3/envs/ml-skl/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [104]:
caracteristics = caracteristics.drop(columns=['adr', 'lat',  'long', 'gps'])

In [105]:
pd.unique(caracteristics['an'])

array([16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5])

In [106]:
users = pd.read_csv(data_folder + 'users.csv', encoding='latin-1')

In [107]:
users = users.drop(columns=['num_veh'])

In [108]:
users.head(3)

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais
0,201600000001,1.0,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0
1,201600000001,1.0,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0
2,201600000002,1.0,1,3,1,5.0,11.0,0.0,0.0,0.0,1960.0


In [109]:
vehicles = pd.read_csv(data_folder + 'vehicles.csv', encoding='latin-1')

In [110]:
vehicles = vehicles.drop(columns=['num_veh', 'senc', 'occutc', 'obs', 'obsm', 'manv', 'choc'])

In [111]:
vehicles.head(3)

,Num_Acc,catv
0,201600000001,7
1,201600000001,2
2,201600000002,7


pour analyser plus finement les données et pour augmenter les nombre les patterns on peut faire certaines pretraitemnt. notament transformer les annnée de naissance en tranche d'age. par exemple tranche d'age entre 18-25, 26-35, 36-50, 51-*.

In [112]:
print(caracteristics.shape)
print(users.shape)

dataset = caracteristics.merge(users, on='Num_Acc')
print(dataset.shape)
dataset = dataset.merge(vehicles, on='Num_Acc')
dataset.shape

(839985, 12)
(1876005, 11)
(1876005, 22)


(3553976, 23)

In [37]:
dataset.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,catv
0,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0,7
1,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0,2
2,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0,7
3,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0,2
4,201600000002,16,3,16,1800,1,2,6,1.0,6.0,...,1,3,1,5.0,11.0,0.0,0.0,0.0,1960.0,7


In [99]:
dataset.shape

(3553976, 24)

In [113]:
caracteristics = None
users = None
vehicles = None

In [39]:
def convert_to_int(val):
    try:
        return int(val)
    except ValueError:
        return -1

In [40]:
age_ranges = {
    1: (0, 12),
    2: (13, 17),
    3: (18, 25),
    4: (26, 35),
    5: (36, 50),
    6: (51, 110)
}

In [41]:
def get_age_range(age_ranges, age):
    for a in age_ranges:
        if a[0] <= age <= a[1]:
            return age_ranges[a]

    raise ValueError(f'invalide age ranges {age}')

def user_age_range(dataset: pd.DataFrame, age_ranges: dict):
    
    ar = {v: k for k, v in age_ranges.items()}
    
    dataset['age_range'] = 0
    
    for i in dataset.index:
        an = convert_to_int(dataset.at[i, 'an'])
        an_nais = convert_to_int(dataset.at[i, 'an_nais'])
        if an == -1 or an_nais == -1: 
            continue
    
        age = (2000 + an) - an_nais
        dataset.at[i, "age_range"] = get_age_range(ar, age)


In [42]:
user_age_range(dataset, age_ranges)

In [43]:
dataset.head(3)

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,catv,age_range
0,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,1,2,0.0,11.0,0.0,0.0,0.0,1983.0,7,4
1,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,1,2,0.0,11.0,0.0,0.0,0.0,1983.0,2,4
2,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,3,1,9.0,21.0,0.0,0.0,0.0,2001.0,7,2


In [44]:
f = open('frequent_info.json', 'r')
data_range = json.load(f)

def key_gen(col_name, val):
    return f'{col_name}__{val}'

def get_unique_id(unique_ids: dict, col_name: str, val):

    vi = convert_to_int(val)

    if col_name in data_range:
        r = data_range[col_name]['range']
        if not (r[0] <= vi <= r[1]):
            vi = -1

    k = key_gen(col_name, vi)

    if k not in unique_ids.keys():
        unique_ids[k] = len(unique_ids.keys())

    return unique_ids[k]


def transform_to_transaction(df: pd.DataFrame):
    unique_ids = {}

    result = add_cols(pd.DataFrame(), df.columns, -1)
    result['Num_Acc'] = df['Num_Acc']

    for col in df.columns:
        if str(col) == 'Num_Acc': continue

        result[str(col)] = df[col].map(lambda x: get_unique_id(unique_ids, str(col), x))

    return result, unique_ids

In [96]:
dataset_tr, unique_ids = transform_to_transaction(dataset)

KeyboardInterrupt: 

In [47]:
len(unique_ids.keys())

1525

In [48]:
dataset_tr.head(3)

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,catv,age_range
0,201600000001,0,1,13,44,495,500,502,511,520,...,1371,1375,1377,1384,1388,1397,1406,1410,1511,1518
1,201600000001,0,1,13,44,495,500,502,511,520,...,1371,1375,1377,1384,1388,1397,1406,1410,1512,1518
2,201600000001,0,1,13,44,495,500,502,511,520,...,1372,1376,1378,1384,1388,1397,1406,1411,1511,1519


In [49]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

te = TransactionEncoder()

In [50]:
dataset_values = dataset_tr.values

In [51]:
dataset_values = dataset_values[:, 1:] # remove Num_Acc

In [52]:
sub_sample_dataset = dataset_values #[:50000]

In [53]:
sub_sample_dataset[:2]

array([[   0,    1,   13,   44,  495,  500,  502,  511,  520,  527, 1264,
        1357, 1367, 1371, 1375, 1377, 1384, 1388, 1397, 1406, 1410, 1511,
        1518],
       [   0,    1,   13,   44,  495,  500,  502,  511,  520,  527, 1264,
        1357, 1367, 1371, 1375, 1377, 1384, 1388, 1397, 1406, 1410, 1512,
        1518]])

In [54]:
te_ary = te.fit(sub_sample_dataset).transform(sub_sample_dataset)

In [55]:
df = pd.DataFrame(te_ary, columns=te.columns_)

In [56]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524
0,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [77]:
import json


def filter_binary_dataset(binary_df: pd.DataFrame, unique_ids:dict, original_df: pd.DataFrame):
    cols = list(original_df.columns)
    to_remove = [ unique_ids[key_gen(col_name, -1)] for col_name in cols if key_gen(col_name, -1) in unique_ids ]
    return binary_df.drop(columns=to_remove)


In [78]:
df = filter_binary_dataset(df, unique_ids, dataset_sub)

In [79]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524
0,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [ ]:
df.to_csv('bin_db.csv', header=False)

In [80]:
from mlxtend.frequent_patterns import association_rules, fpmax, fpgrowth, apriori, fpmax

In [84]:
fpmax_set = fpgrowth(df, min_support=0.2, use_colnames=True)
fpmax_set_assoc = association_rules(fpmax_set, min_threshold=0.4)

### alternatively:
#frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
#frequent_itemsets = fpmax(df, min_support=0.6, use_colnames=True)

In [97]:
fpmax_set_assoc.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(0),(1397),1.00000,0.96968,0.96968,0.969680,1.000000,0.000000,1.000000
1,(1397),(0),0.96968,1.00000,0.96968,1.000000,1.000000,0.000000,inf
2,(0),(502),1.00000,0.79700,0.79700,0.797000,1.000000,0.000000,1.000000
3,(502),(0),0.79700,1.00000,0.79700,1.000000,1.000000,0.000000,inf
4,(1397),(502),0.96968,0.79700,0.77150,0.795623,0.998273,-0.001335,0.993264


In [86]:
id_inv_map = {v: k for k, v in unique_ids.items()}

In [87]:
def decompose_key(k:str):
    return k.split('__')[0], int(k.split('__')[1])

In [90]:
# save resulte

fpmax_set.to_csv('closed_itemset.csv', index=False)
fpmax_set_assoc.to_csv('closed_itemset.csv', index=False)

with open('id_inv_map.json', 'w') as f:
    json.dump(id_inv_map, f)


In [94]:
f = open('descritption.json', 'r')
desc = json.load(f)

res = []
for i, itemset in fpmax_set.iterrows():
    l = [itemset['support']]
    
    it = []
    for i in itemset['itemsets']:
        col_name , val = decompose_key(id_inv_map[i])
        d = '___'
        col_desc = desc[col_name]['description']
        
        if col_desc['type'] == 'mono':
            d = col_desc['value']
        else:
            d = col_desc[str(val)]
        
        it.append((col_name, val, d))
        
    res.append((it, l))


In [95]:
s = 0
for r in res: #[s:s + 10]:
    print(r)
    print()

([('an', 16, "Anne de l'accident")], [1.0])

([('actp', 0, 'non spécifié ou non applicable')], [0.96968])

([('int', 1, 'Hors intersection')], [0.797])

([('lum', 1, 'Plein Jour')], [0.73918])

([('place', 1, 'Place occupée')], [0.73888])

([('catu', 1, 'Pilote')], [0.73878])

([('catv', 7, 'VL uniquement')], [0.6702])

([('grav', 1, 'Indemne')], [0.38546])

([('sexe', 2, 'Femme')], [0.3309])

([('agg', 2, 'En agglomération')], [0.31974])

([('sexe', 1, 'Homme')], [0.6691])

([('grav', 3, 'Blessés hospitalisés')], [0.33792])

([('atm', 1, 'Normal')], [0.75308])

([('trajet', 5, 'Promenade - loisirs')], [0.41596])

([('age_range', 6, '51 - 110')], [0.32548])

([('agg', 1, 'Hors agglomération')], [0.68026])

([('an', 16, "Anne de l'accident"), ('actp', 0, 'non spécifié ou non applicable')], [0.96968])

([('an', 16, "Anne de l'accident"), ('int', 1, 'Hors intersection')], [0.797])

([('actp', 0, 'non spécifié ou non applicable'), ('int', 1, 'Hors intersection')], [0.7715])

([('an', 16, "